# Vježba 3 - Vektorska reprezentacija dokumenta

<ol><li> Kreirati promjenljivu <b>str1</b> koja sadrži tekst 'zdravo', te listu <b>str_list</b> koja sadrži stringove: 'zdravo', 'zbogom', 'zdravo', 'zbogom', 'zdravo', 'zdravoratumski'. Napisati kod koji broji koliko puta se string 'zdravo' nalazi u listi <b>str_list</b>.
</li></ol>

In [1]:
#KOD ZA ZADATAK 1
str1 = 'zdravo'
str_list = ['zdravo', 'zbogom', 'zdravo', 'zbogom', 'zdravo', 'zdravoratumski']
cnt = [str.count(str1) for str in str_list]
print(sum(cnt))

4


<ol start = "2">
<li> Napisati kod koji se u listi <b>str_list</b> pronalazi string najsličniji riječi 'zdravorazumski', pri čemu se sličnost mjeri brojem istih slova.
</li>
</ol>

In [2]:
#KOD ZA ZADATAK 2
qry = 'zdravorazumski'
sim = [len(set(qry).intersection(str)) for str in str_list]
print(str_list[sim.index(max(sim))], max(sim))

zdravoratumski 11


<ol start = "3">
<li> Kreirajte promjenljivu koja sadrži string 'eci peci pec ti si mali zec'. Tokenizujte ovu promjenljivu na riječi koje sadrži. Rezultat treba da bude lista koja sadrži pojedine riječi zadatog stringa.
</li>
</ol>

In [3]:
#KOD ZA ZADATAK 3
str = 'eci peci pec ti si mali zec'
l = str.split()
print(l)


['eci', 'peci', 'pec', 'ti', 'si', 'mali', 'zec']


U narednom dijelu vježbe ćemo iskoristiti korpus dokumenata sa projekta Gutenberg da bismo kreirali njihovu vektorsku reprezentaciju, a nakon toga implementirali sistem koji za proizvoljan dokument pronalazi najsličniji dokument iz Gutenberg korpusa. U narednoj ćeliji koda su date dvije funkcije koje se mogu iskoristiti za tokenizaciju dokumenta.

In [4]:
from nltk.stem.porter import PorterStemmer
import re
import nltk

stemmer = PorterStemmer()
minlen = 1

def normalize(word):
    # Funkcija koja vrši normalizaciju riječi
    word = re.sub('[^A-Za-z0-9]+', '', word)    #Ukljanjanje svih karakter koji nisu alfanumerički
    word = re.sub('[0-9]+', 'number', word)     #Zamjena svih brojeva u tekstu riječju number
    word = word.lower()                         #Pretvaranje u mala slova
    return word

def tokenize(text):
    #Funkcija koja vrši tokenizaciju teksta na sastavne riječi
    tokens = nltk.word_tokenize(text)           #Tokenizacija teksta
    stems = []
    for token in tokens:
        word = normalize(token)                 #Normalizacija pronađenih tokena
        stem = stemmer.stem(word)               #Stemizacija tokena
        if len(stem) > minlen:                  #Zadržavanje tokena čija je dužina veća od minlen
            stems.append(stem)
    return stems

Potrebno je napisati kod koji će za sve dokumente iz Gutenberg korpusa izračunati njihovu vektorsku reprezentaciju i to sačuvati u promjenljivu representations. Da bi se to uradilo potrebno je:
1. Za svaki dokument odrediti tokene,
2. Odrediti rječnik za dati korpus,
3. Za svaki dokument na osnovu rječnika odrediti vektorsku reprezentaciju.
U narednoj ćeliji su date kostur potrebnog koda.

In [5]:
import os
from nltk.corpus import stopwords

#Putanja do korpusa
path = './gutenberg'

#Dictionary koji će za svaki dokument sadržati njegove tokene
token_dict = {}

#Prolazak kroz sve fajlove
for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)    #Putanja do fajla
        #Izršiti učitavanje fajla i njegovu tokenizaciju
        with open(fname, 'r') as fh:
            txt = fh.read()
        #Tokene svakog fajla upisati u token_dict i to tako da je ključ naziv fajla, a vrijednost tokeni
        token_dict[fname] = tokenize(txt)

#Lista koja će predstavljati rječnik svih rječi iz korpusa
dictionary = []
stop_words = set(stopwords.words('english'))

#Proći kroz tokene za sve fajlove i dodati ih u dictionary, obratiti pažnju da često korištene riječi (stop_words) nije potrebno
#dodavati u rječnik

for tk_list in token_dict.values():
    dictionary += [term for term in tk_list if term not in stop_words]
            
#Kako rječnik ne bi trebalo da sadrži ponovljene riječi potrebno je ostaviti samo jedinstvene riječi

dictionary = set(dictionary)

#Kako bi se ubrzalo pretraživanje rječnika, rječnik se pretvara u dictionary koji za ključeve ima riječi, a za vrijednosti
#indeks koji odgovara poziciji te riječi u reprezentaciji

dictionary_dict = {word: i for i, word in enumerate(dictionary)}

print(len(dictionary_dict.keys()))

29670


In [6]:
import numpy as np

def compute_document_representation(tokens):
    #Napisati funkciju koja kreira reprezentaciju dokumenta na osnovu ulaznih tokena i to tako da se za svaku riječ iz rječnika
    #odredi koliko puta se pojavljuje u dokumentu
    #Izlaz treba biti niz ranga 1 dužine koja odgovara dužini rječnika
    
    vec = np.zeros(len(dictionary_dict.keys()))
    for t in tokens:
        if t in dictionary_dict:
            vec[dictionary_dict[t]] += 1
    return vec
    
#Kreiranje numpy niza koji će sadržati reprezentacije svakog dokumenta iz korpusa, shape je (broj_dokumenata, broj_rječi)
representations = np.zeros((len(token_dict), len(dictionary_dict)))

#Kreirati reprezentaciju svih dokumenta iz korpusa tako što ćete iskoristiti prethodno napisanu funkciju

for k, doc in enumerate(token_dict.keys()):
    print(doc)
    representations[k] = compute_document_representation(token_dict[doc])


./gutenberg\austen-emma.txt
./gutenberg\austen-persuasion.txt
./gutenberg\austen-sense.txt
./gutenberg\bible-kjv.txt
./gutenberg\blake-poems.txt
./gutenberg\bryant-stories.txt
./gutenberg\burgess-busterbrown.txt
./gutenberg\carroll-alice.txt
./gutenberg\chesterton-ball.txt
./gutenberg\chesterton-brown.txt
./gutenberg\chesterton-thursday.txt
./gutenberg\edgeworth-parents.txt
./gutenberg\melville-moby_dick.txt
./gutenberg\milton-paradise.txt
./gutenberg\shakespeare-caesar.txt
./gutenberg\shakespeare-hamlet.txt
./gutenberg\shakespeare-macbeth.txt
./gutenberg\whitman-leaves.txt


Nakon što smo odredili reprezentaciju za sve dokumente ove reprezentacije je moguće iskoristiti za pretraživanje. Pretraživanje izvršiti na način da se za proizvoljno izabran dokument pronalaze tri najsličnija dokumenta iz korpusa. Za računanje sličnosti iskoristiti i SSD i kosinusnu sličnost. Za računanje kosinusne sličnosti je moguće iskoristiti funkciju cosine_similarity iz modula sklearn.metrics.pairwise.

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

csim = cosine_similarity(representations[0].reshape(1, -1), representations)
idx = csim.argsort()
print(idx[0][-2:-5:-1])
print(list(token_dict.keys())[0])
for i in idx[0][-2:-5:-1]:
    print(list(token_dict.keys())[i])


[ 1  2 11]
./gutenberg\austen-emma.txt
./gutenberg\austen-persuasion.txt
./gutenberg\austen-sense.txt
./gutenberg\edgeworth-parents.txt


Za računanje TFIDF reprezentacije dokumenta moguće je iskoristiti sklearn biblioteku i kod dat u narednoj ćeliji.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string

#Putanja do korpusa
path = './gutenberg'

#Dictionary koji sadrži tekst svakog dokumenta
text_dict = {}

#Čitanje sadržaja svih dokumenata
for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)
        print (fname)
        with open(fname) as file:
            text_dict[f] = file.read()

#Definisanje objekta koji služi za određivanje TFIDF reprezentacije, za tokenizaciju se koristifunkcije data
#kao parametar tokenize, dok se kao stop riječi koriste one iz enegleskog jezika (moguće je zadati i listu koja sadrži stop riječi)
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

#Učenje rječnika i transformacija dokumenata koji su iskorišteni za učenje rječnika
#U representations_tfidf se sada nalazi rezrezentacija svih dokumenata iz korpusa
representations_tfidf = tfidf.fit_transform(text_dict.values())

str = 'all great and precious things are lonely.'

#Određivanje reprezentacije za novi dokument
#Treba obratiti pažnju da su reprezentacije date kao sparse matrice radi efikasnijg čuvanja u memoriji ali i bržeg procesiranja.
response = tfidf.transform([str])

response

./gutenberg\austen-emma.txt
./gutenberg\austen-persuasion.txt
./gutenberg\austen-sense.txt
./gutenberg\bible-kjv.txt
./gutenberg\blake-poems.txt
./gutenberg\bryant-stories.txt
./gutenberg\burgess-busterbrown.txt
./gutenberg\carroll-alice.txt
./gutenberg\chesterton-ball.txt
./gutenberg\chesterton-brown.txt
./gutenberg\chesterton-thursday.txt
./gutenberg\edgeworth-parents.txt
./gutenberg\melville-moby_dick.txt
./gutenberg\milton-paradise.txt
./gutenberg\shakespeare-caesar.txt
./gutenberg\shakespeare-hamlet.txt
./gutenberg\shakespeare-macbeth.txt
./gutenberg\whitman-leaves.txt


<1x29559 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

Za proizvoljan dokument odrediti 3 najsličnija dokumenta iz korpusa na osnovu SSD i kosinusne sličnosti. Prethodno navedenu funkciju za kosinusnu sličnost je moguće koristiti i sa sparse matricama.

In [10]:
csim = cosine_similarity(representations_tfidf[0], representations_tfidf)
idx = csim.argsort()
print(idx)
print(idx[0][-2:-5:-1])
print(list(token_dict.keys())[0])
for i in idx[0][-2:-5:-1]:
    print(list(token_dict.keys())[i])

[[14 16  3 15  4  6 13  7 17 10  8 12  5  2  9  1 11  0]]
[11  1  9]
./gutenberg\austen-emma.txt
./gutenberg\edgeworth-parents.txt
./gutenberg\austen-persuasion.txt
./gutenberg\chesterton-brown.txt
